<div style="text-align:center"><h1>Running Coupling Constant</h1></div>
<div style="text-align:right"><b>Tae Geun Kim</b></div>
*****
* Caution : This is test version, so consider only 1-loop

In [1]:
from numba import jit

In [9]:
import numpy as np
from math import sqrt, exp, log, log10, pi

class RGE(object):
        
    def __init__(self, mt_init, xi_init, tstart):
        self.t = tstart
        self.mt = mt_init
        self.xi = xi_init
        self.Mp = 1.221*10**(19)
        self.MpR = 2.4*10**(18) # Reduced Planck Mass
        self.MW = 80.385
        self.MZ = 91.1876
        self.MH = 125.09
        self.alphasMZ = 0.1182
        self.lH = 0.12604 + 0.00206 * (self.MH - 125.15) - 0.00004 * (self.mt - 173.34)
        self.yt = (
            0.93690 + 0.00556 * (self.mt - 173.34) - 0.00003*(self.MH - 125.15) 
            - 0.00042*(self.alphasMZ - 0.1184) / 0.0007
        )
        self.g3 = 1.1666 + 0.00314 * (self.alphasMZ - 0.1184)/0.007 - 0.00046 * (self.mt - 173.34)
        self.g2 = 0.64779 + 0.00004 * (self.mt - 173.34) + 0.00011 * (self.MW - 80.384) / 0.014
        self.g1 = 0.35830 + 0.00011 * (self.mt - 173.34) - 0.00020 * (self.MW - 80.384) / 0.014
    
    def BETA(self):
        # 1-Loop Beta Functions
        self.h = sqrt(2)/self.yt * self.mt * exp(self.t)
        self.sh = (
            (1 + self.xi * self.h ** 2 / self.MpR**2)/(1+(1+6*self.xi)*self.xi*self.h**2/self.MpR**2)
        )
        self.Beta_lH1 = (
            6*(1+3*self.sh**2)*self.lH**2 + 12*self.lH*self.yt**2 - 6*self.yt**4 - 3*self.lH*(3*self.g2**2 + self.g1**2)
            + 3/8*(2*self.g2**4 + (self.g1**2 + self.g2**2)**2)
        )
        self.Beta_g11 = (81 + self.sh)/12 * self.g1**3
        self.Beta_g21 = - (39 - self.sh) / 12 * self.g2 **3
        self.Beta_g31 = -7*self.g3**3
        self.Beta_yt1 = (
            self.yt * ((23/6 + 2/3*self.sh)*self.yt**2 - (8*self.g3**2 + 9/4*self.g2**2 + 17/12 * self.g1**2))
        )
        self.gamma1 = -(9/4*self.g2**2 + 3/4*self.g1**2 - 3*self.yt**2)
        self.Betas1 = [self.Beta_lH1, self.Beta_g11, self.Beta_g21, 
                       self.Beta_g31, self.Beta_yt1, self.gamma1]
        # 2-Loop Beta Functions
        self.Beta_lH2 = 0
        self.Beta_g12 = 0
        self.Beta_g22 = 0
        self.Beta_g32 = 0
        self.Beta_yt2 = 0
        self.gamma2 = 0
        self.Betas2 = [self.Beta_lH2, self.Beta_g12, self.Beta_g22, 
                       self.Beta_g32, self.Beta_yt2, self.gamma2]
        # Total Beta Functions
        self.Beta = np.zeros(len(self.Betas1))
        for i, (beta1, beta2) in enumerate(zip(self.Betas1, self.Betas2)):
            self.Beta[i] = 1/(16*pi**2)*beta1 + 1/(16*pi**2)**2*beta2
        self.Beta_real = [beta/(1+self.Beta[5]) for beta in self.Beta]
        
    @jit   
    def Running(self, h=1e-05):
        # Running Coupling Constant
        self.lH += h * self.Beta_real[0]
        self.g1 += h * self.Beta_real[1]
        self.g2 += h * self.Beta_real[2]
        self.g3 += h * self.Beta_real[3]
        self.yt += h * self.Beta_real[4]
        self.t += h
#-----------------------------------------------     
def RCC(Mt, xi, Nend, h): # M_t, xi, End point, precision
    # Real Running
    A = RGE(Mt, xi, 0) # xi = 59
    k = int(1/h * Nend)
    lH, g1, g2, g3 = np.zeros(k),np.zeros(k),np.zeros(k),np.zeros(k)
    yt, t = np.zeros(k), np.zeros(k)
    for i in range(k):
        A.BETA()
        lH[i] = A.lH
        g1[i] = A.g1
        g2[i] = A.g2
        g3[i] = A.g3
        yt[i] = A.yt
        t[i] = A.t
        A.Running(h)
    return (lH, g1, g2, g3, yt, t)

In [10]:
%%timeit 
RCC(170, 50, 10, 1e-05)

1 loop, best of 3: 35.5 s per loop


In [ ]:
lH = Temp[0]
g1 = Temp[1]
g2 = Temp[2]
g3 = Temp[3]
yt = Temp[4]
t = Temp[5]